## imports and files

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    PIPELINE_PROFILING_CONFIGS_PATH,
    PIPELINE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

In [2]:
from json import load

series = 13
config_key_mapper = "key_config_mapper.csv"

series_path = os.path.join(PIPELINE_PROFILING_RESULTS_PATH, "series", str(series))

loader = Parser(
    series_path=series_path,
    config_key_mapper=config_key_mapper,
    model_name="video",
    type_of="pipeline",
)

In [3]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 0.yaml
{   'benchmark_duration': 1,
    'metadata': 'Trying out the pipeline',
    'mode': 'exponential',
    'nodes': [   {   'cpu_request': ['1', '2'],
                     'data_type': 'image',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['yolov5n'],
                     'node_name': 'yolo',
                     'num_interop_threads': ['1'],
                     'num_threads': ['1'],
                     'replicas': [1],
                     'use_threading': 'True'},
                 {   'cpu_request': ['1', '2'],
                     'data_type': 'text',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['resnet18'],
                     'node_name': 'resnet-human',
                     'num_interop_threads': 

## config map reader

In [4]:
key_config_df = loader.key_config_mapper()
display(key_config_df)
key_config_df.keys()

,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,1,1,1
1,2,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,2,10Gi,1,1,1
2,3,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,1,1,1
3,4,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,2,10Gi,1,1,1


Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')

In [5]:
experiment_results = loader.get_experiment_detail(1)
display(experiment_results)
experiment_results.keys()

,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,1,1,1


Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')

In [6]:
filenames = loader.get_result_file_names()
filenames

['4.json', '1.json', '3.json', '2.json']

## Read resutls

In [7]:
results = loader._read_results()
print(results.keys())

dict_keys(['4', '1', '3', '2'])


In [8]:
flattened_results = loader.flatten_results(results["1"]["responses"])
flattened_results

[{'times': {'request': {'sending': 1679858699.3923442,
    'arrival': 1679858699.7178962},
   'models': {'resnet-human': {'arrival': 1679858699.536556,
     'serving': 1679858699.6785338},
    'yolo': {'arrival': 1679858699.4245467, 'serving': 1679858699.5135128}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1679858700.8542635,
    'arrival': 1679858701.0801153},
   'models': {'resnet-human': {'arrival': 1679858700.988693,
     'serving': 1679858701.06292},
    'yolo': {'arrival': 1679858700.8802571, 'serving': 1679858700.9655416}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1679858701.061981,
    'arrival': 1679858701.2840617},
   'models': {'resnet-human': {'arrival': 1679858701.2020047,
     'serving': 1679858701.2684755},
    'yolo': {'arrival': 1679858701.0984125, 'serving': 1679858701.1796107}}},
  'model_name': 'resnet-human',
  'outputs': []},
 {'times': {'request': {'sending': 1679858703.13302

In [9]:
experiment_id = 1
flattened_results = loader.flatten_results(results[str(experiment_id)]["responses"])
latencies, timeout_count = loader.latency_calculator(flattened_results)
print(f"{timeout_count=}")
print(pd.DataFrame(latencies))

timeout_count=0
    client_to_pipeline_latencies  pipeline_to_client_latencies  \
0                       0.032202                      0.039362   
1                       0.025994                      0.017195   
2                       0.036432                      0.015586   
3                       0.028498                      0.009985   
4                       0.022036                      0.008456   
5                       0.037820                      0.013665   
6                       0.029393                      0.009618   
7                       0.028507                      0.011700   
8                       0.026557                      0.010306   
9                       0.025382                      0.011759   
10                      0.027100                      0.008735   
11                      0.107675                      0.010939   
12                      0.027426                      0.008399   
13                      0.023393                      0.0130

In [10]:
summary = loader.latency_summary(latencies=latencies)
pd.DataFrame(list(summary.items()))

,0,1
0,client_to_pipeline_latencies_avg,0.035250
1,client_to_pipeline_latencies_p99,0.126494
2,client_to_pipeline_latencies_p50,0.026779
3,client_to_pipeline_latencies_var,0.000585
4,client_to_pipeline_latencies_max,0.134181
5,client_to_pipeline_latencies_min,0.022036
6,pipeline_to_client_latencies_avg,0.012255
7,pipeline_to_client_latencies_p99,0.032934
8,pipeline_to_client_latencies_p50,0.011359
9,pipeline_to_client_latencies_var,0.000029


In [11]:
results = loader.result_processing()
# print(results)
# print(list(results.columns))
for item in list(results.columns):
    if "task_0" in item:
        print(item)

task_0_model_latencies_avg
task_0_model_latencies_p99
task_0_model_latencies_p50
task_0_model_latencies_var
task_0_model_latencies_max
task_0_model_latencies_min
task_0_to_task_1_latencies_avg
task_0_to_task_1_latencies_p99
task_0_to_task_1_latencies_p50
task_0_to_task_1_latencies_var
task_0_to_task_1_latencies_max
task_0_to_task_1_latencies_min
task_0_cpu_usage_count_avg
task_0_cpu_usage_count_p99
task_0_cpu_usage_count_p50
task_0_cpu_usage_count_var
task_0_cpu_usage_count_max
task_0_cpu_usage_count_min
task_0_cpu_usage_rate_avg
task_0_cpu_usage_rate_p99
task_0_cpu_usage_rate_p50
task_0_cpu_usage_rate_var
task_0_cpu_usage_rate_max
task_0_cpu_usage_rate_min
task_0_cpu_throttled_count_avg
task_0_cpu_throttled_count_p99
task_0_cpu_throttled_count_p50
task_0_cpu_throttled_count_var
task_0_cpu_throttled_count_max
task_0_cpu_throttled_count_min
task_0_cpu_throttled_rate_avg
task_0_cpu_throttled_rate_p99
task_0_cpu_throttled_rate_p50
task_0_cpu_throttled_rate_var
task_0_cpu_throttled_rate_ma

In [12]:
key_config_df = loader.key_config_mapper()
print(key_config_df.columns)
# selected_key_config_df =  key_config_df[key_config_df['model_variant_1'] == 'yolov5n']
# selected_key_config_df
key_config_df

Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')


,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,1,1,1
1,2,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,2,10Gi,1,1,1
2,3,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,1,1,1
3,4,video,1,30,13,Trying out the pipeline,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,2,10Gi,1,1,1


In [13]:
results.columns

Index(['experiment_id', 'client_to_pipeline_latencies_avg',
       'client_to_pipeline_latencies_p99', 'client_to_pipeline_latencies_p50',
       'client_to_pipeline_latencies_var', 'client_to_pipeline_latencies_max',
       'client_to_pipeline_latencies_min', 'pipeline_to_client_latencies_avg',
       'pipeline_to_client_latencies_p99', 'pipeline_to_client_latencies_p50',
       ...
       'task_1_memory_usage_p50', 'task_1_memory_usage_var',
       'task_1_memory_usage_max', 'task_1_memory_usage_min',
       'task_1_throughput_avg', 'task_1_throughput_p99',
       'task_1_throughput_p50', 'task_1_throughput_var',
       'task_1_throughput_max', 'task_1_throughput_min'],
      dtype='object', length=107)

In [14]:
experiment_ids = key_config_df[key_config_df["task_0_node_name"] == "yolo"][
    "experiment_id"
].tolist()
# model = 'resnet-human'
# pod = 'pod1'
task = "task_0"
metadata_columns = ["task_0_node_name", "task_0_cpu_request"]
results_columns = ["client_to_pipeline_latencies_p99", f"{task}_model_latencies_p99"]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
display(output)

,task_0_node_name,task_0_cpu_request,client_to_pipeline_latencies_p99,task_0_model_latencies_p99
0,yolo,1,0.126494,0.103461
1,yolo,1,0.120139,0.115492
2,yolo,2,0.063930,0.101587
3,yolo,2,0.045538,0.091135


In [15]:
for item in results.columns:
    print(item)

experiment_id
client_to_pipeline_latencies_avg
client_to_pipeline_latencies_p99
client_to_pipeline_latencies_p50
client_to_pipeline_latencies_var
client_to_pipeline_latencies_max
client_to_pipeline_latencies_min
pipeline_to_client_latencies_avg
pipeline_to_client_latencies_p99
pipeline_to_client_latencies_p50
pipeline_to_client_latencies_var
pipeline_to_client_latencies_max
pipeline_to_client_latencies_min
task_0_model_latencies_avg
task_0_model_latencies_p99
task_0_model_latencies_p50
task_0_model_latencies_var
task_0_model_latencies_max
task_0_model_latencies_min
task_0_to_task_1_latencies_avg
task_0_to_task_1_latencies_p99
task_0_to_task_1_latencies_p50
task_0_to_task_1_latencies_var
task_0_to_task_1_latencies_max
task_0_to_task_1_latencies_min
task_1_model_latencies_avg
task_1_model_latencies_p99
task_1_model_latencies_p50
task_1_model_latencies_var
task_1_model_latencies_max
task_1_model_latencies_min
start_time
end_time
duration
timeout_count
task_0_cpu_usage_count_avg
task_0_cpu

In [33]:
for item in results.columns:
    print(item)

experiment_id
client_to_pipeline_latencies_avg
client_to_pipeline_latencies_p99
client_to_pipeline_latencies_p50
client_to_pipeline_latencies_var
client_to_pipeline_latencies_max
client_to_pipeline_latencies_min
pipeline_to_client_latencies_avg
pipeline_to_client_latencies_p99
pipeline_to_client_latencies_p50
pipeline_to_client_latencies_var
pipeline_to_client_latencies_max
pipeline_to_client_latencies_min
task_0_model_latencies_avg
task_0_model_latencies_p99
task_0_model_latencies_p50
task_0_model_latencies_var
task_0_model_latencies_max
task_0_model_latencies_min
task_0_to_task_1_latencies_avg
task_0_to_task_1_latencies_p99
task_0_to_task_1_latencies_p50
task_0_to_task_1_latencies_var
task_0_to_task_1_latencies_max
task_0_to_task_1_latencies_min
task_1_model_latencies_avg
task_1_model_latencies_p99
task_1_model_latencies_p50
task_1_model_latencies_var
task_1_model_latencies_max
task_1_model_latencies_min
start_time
end_time
duration
timeout_count
task_0_cpu_usage_count_avg
task_0_cpu